In [10]:
#Note for self; This appear to be the algorithm for the greedy implementation where next experiment is determined based on the best predicted value and lowest variance (also utilize best feature importance). 

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.externals.joblib import dump

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [16]:
filename = 'train_data.txt'
train = pd.read_csv(filename, sep= '\t')
array = train.values
X = array[:,1:-1] 
Y = array[:,-1]

df

,0,1,2,3,4,5,6
0,0.1,0.1,0.1,10,0.1,0.0,5
1,0.1,0.1,0.1,10,0.1,0.0,10
2,0.1,0.1,0.1,10,0.1,0.0,15
3,0.1,0.1,0.1,10,0.1,0.0,30
4,0.1,0.1,0.1,10,0.1,0.0,60
...,...,...,...,...,...,...,...
26995,0.3,0.3,0.3,80,1.5,10.0,5
26996,0.3,0.3,0.3,80,1.5,10.0,10
26997,0.3,0.3,0.3,80,1.5,10.0,15
26998,0.3,0.3,0.3,80,1.5,10.0,30


In [17]:
X

array([[ 0.1 ,  0.2 ,  0.1 , 80.  ,  1.5 ,  0.  , 60.  ],
       [ 0.2 ,  0.1 ,  0.1 , 10.  ,  0.1 ,  4.  , 15.  ],
       [ 0.3 ,  0.1 ,  0.3 , 10.  ,  1.5 ,  3.  ,  5.  ],
       [ 0.3 ,  0.2 ,  0.1 , 40.  ,  0.1 ,  3.  , 10.  ],
       [ 0.3 ,  0.1 ,  0.2 , 40.  ,  0.1 ,  2.  , 15.  ],
       [ 0.2 ,  0.3 ,  0.2 , 20.  ,  1.  ,  3.  ,  5.  ],
       [ 0.1 ,  0.1 ,  0.2 , 60.  ,  1.5 ,  2.  ,  5.  ],
       [ 0.3 ,  0.2 ,  0.1 , 40.  ,  0.25,  1.  ,  5.  ],
       [ 0.1 ,  0.2 ,  0.3 , 60.  ,  0.5 ,  4.  ,  5.  ],
       [ 0.2 ,  0.3 ,  0.1 , 10.  ,  1.  ,  4.  ,  5.  ]])

In [18]:
Y

array([1.2, 2.9, 2.2, 7. , 4.2, 3.1, 1.2, 1.9, 6.7, 4.7])

In [19]:
seed = 1  
kfold = KFold(n_splits = 10, shuffle=True, random_state = seed)
scoring = 'neg_mean_absolute_error'
model = RandomForestRegressor(random_state=seed)
estimators = np.arange(100, 1050, 50)

In [20]:
estimators 

array([ 100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  800,  850,  900,  950, 1000])

In [22]:
estimators_int = np.ndarray.tolist(estimators)

In [23]:
param_grid = {'n_estimators':estimators_int, 'max_features':('auto', 'sqrt'), 'max_depth':[None, 2, 4]}

In [24]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, n_jobs=6)
grid_result = grid.fit(X, Y)
np.savetxt('best_score.txt', ["best_score: %s" % grid.best_score_], fmt ='%s')
best_params = pd.DataFrame([grid.best_params_], columns=grid.best_params_.keys())

In [25]:
filename2 = 'all_combos.txt'
df_all_combos = pd.read_csv(filename2, sep= '\t')
df_all_combos 

,Unnamed: 0,Pyridine,Aldehyde,Isocyanide,Temperature,Solvent,Catalyst,Time
0,0,0.1,0.1,0.1,10,0.1,0.0,5
1,1,0.1,0.1,0.1,10,0.1,0.0,10
2,2,0.1,0.1,0.1,10,0.1,0.0,15
3,3,0.1,0.1,0.1,10,0.1,0.0,30
4,4,0.1,0.1,0.1,10,0.1,0.0,60
...,...,...,...,...,...,...,...,...
26995,26995,0.3,0.3,0.3,80,1.5,10.0,5
26996,26996,0.3,0.3,0.3,80,1.5,10.0,10
26997,26997,0.3,0.3,0.3,80,1.5,10.0,15
26998,26998,0.3,0.3,0.3,80,1.5,10.0,30


In [38]:
df_train_corrected = train.iloc[:,:-1]
df_train_corrected = df_train_corrected.rename(columns={'Solvent_V': 'Solvent'})

In [39]:
unseen = pd.concat([df_all_combos, df_train_corrected]).drop_duplicates(keep=False)
unseen

,Unnamed: 0,Pyridine,Aldehyde,Isocyanide,Temperature,Solvent,Catalyst,Time
0,0,0.1,0.1,0.1,10,0.1,0.0,5
1,1,0.1,0.1,0.1,10,0.1,0.0,10
2,2,0.1,0.1,0.1,10,0.1,0.0,15
3,3,0.1,0.1,0.1,10,0.1,0.0,30
4,4,0.1,0.1,0.1,10,0.1,0.0,60
...,...,...,...,...,...,...,...,...
26995,26995,0.3,0.3,0.3,80,1.5,10.0,5
26996,26996,0.3,0.3,0.3,80,1.5,10.0,10
26997,26997,0.3,0.3,0.3,80,1.5,10.0,15
26998,26998,0.3,0.3,0.3,80,1.5,10.0,30


In [41]:
array2 = unseen.values
array2

array([[0.0000e+00, 1.0000e-01, 1.0000e-01, ..., 1.0000e-01, 0.0000e+00,
        5.0000e+00],
       [1.0000e+00, 1.0000e-01, 1.0000e-01, ..., 1.0000e-01, 0.0000e+00,
        1.0000e+01],
       [2.0000e+00, 1.0000e-01, 1.0000e-01, ..., 1.0000e-01, 0.0000e+00,
        1.5000e+01],
       ...,
       [2.6997e+04, 3.0000e-01, 3.0000e-01, ..., 1.5000e+00, 1.0000e+01,
        1.5000e+01],
       [2.6998e+04, 3.0000e-01, 3.0000e-01, ..., 1.5000e+00, 1.0000e+01,
        3.0000e+01],
       [2.6999e+04, 3.0000e-01, 3.0000e-01, ..., 1.5000e+00, 1.0000e+01,
        6.0000e+01]])

In [44]:
array2 = unseen.values
X2 = array2[:,1:]
X2.shape

(26990, 7)

In [45]:
df_all_combos2 = df_all_combos.iloc[:,1:]

In [48]:
df_all_combos2

,Pyridine,Aldehyde,Isocyanide,Temperature,Solvent,Catalyst,Time
0,0.1,0.1,0.1,10,0.1,0.0,5
1,0.1,0.1,0.1,10,0.1,0.0,10
2,0.1,0.1,0.1,10,0.1,0.0,15
3,0.1,0.1,0.1,10,0.1,0.0,30
4,0.1,0.1,0.1,10,0.1,0.0,60
...,...,...,...,...,...,...,...
26995,0.3,0.3,0.3,80,1.5,10.0,5
26996,0.3,0.3,0.3,80,1.5,10.0,10
26997,0.3,0.3,0.3,80,1.5,10.0,15
26998,0.3,0.3,0.3,80,1.5,10.0,30


In [49]:
model2 = RandomForestRegressor(n_estimators = grid.best_params_['n_estimators'], max_features = grid.best_params_['max_features'], max_depth = grid.best_params_['max_depth'], random_state = seed)
RF_fit = model2.fit(X, Y)
predictions = model2.predict(X2)
predictions_df = pd.DataFrame(data=predictions, columns=['Prediction'])
feat_imp = pd.DataFrame(model2.feature_importances_, index=list(df_all_combos2.columns.values), columns=['Feature_importances'])
feat_imp = feat_imp.sort_values(by=['Feature_importances'], ascending = False)

In [50]:
feat_imp

,Feature_importances
Solvent,0.319559
Catalyst,0.179172
Temperature,0.142349
Pyridine,0.124440
Time,0.088087
Isocyanide,0.075038
Aldehyde,0.071356


In [51]:
all_predictions = []
for e in model2.estimators_:
    all_predictions += [e.predict(X2)]

variance = np.var(all_predictions, axis=0)
variance_df = pd.DataFrame(data=variance, columns=['Variance'])

assert len(variance) == len(predictions) # control line
initial_data = pd.DataFrame(data=array2, columns = list(unseen.columns.values))
df = pd.concat([initial_data, predictions_df, variance_df], axis=1)
df

,Unnamed: 0,Pyridine,Aldehyde,Isocyanide,Temperature,Solvent,Catalyst,Time,Prediction,Variance
0,0.0,0.1,0.1,0.1,10.0,0.1,0.0,5.0,3.662676,4.059621
1,1.0,0.1,0.1,0.1,10.0,0.1,0.0,10.0,3.789819,4.337245
2,2.0,0.1,0.1,0.1,10.0,0.1,0.0,15.0,3.601800,3.837622
3,3.0,0.1,0.1,0.1,10.0,0.1,0.0,30.0,3.601800,3.837622
4,4.0,0.1,0.1,0.1,10.0,0.1,0.0,60.0,3.320467,3.682375
...,...,...,...,...,...,...,...,...,...,...
26985,26995.0,0.3,0.3,0.3,80.0,1.5,10.0,5.0,3.555729,4.294886
26986,26996.0,0.3,0.3,0.3,80.0,1.5,10.0,10.0,3.751371,4.730377
26987,26997.0,0.3,0.3,0.3,80.0,1.5,10.0,15.0,3.657938,4.607468
26988,26998.0,0.3,0.3,0.3,80.0,1.5,10.0,30.0,3.657938,4.607468


In [53]:
feat_imp_T = feat_imp.transpose() # creates a table with a single row stating the importance (0-1 scale) of each variable
keys1 = list(feat_imp_T.keys()) # collects the names of the features
keys2 = list(feat_imp_T.keys()) # same as above
keys1.insert(7,'Prediction') # Inserts "Prediction" in position 7 of the previously generated list
keys2.insert(7, 'Variance') # Inserts "Variance" in position 7 of the previously generated list

df_sorted = df.sort_values(by=[keys1[-1], keys1[0]], ascending=[False, False]) # Fetches the table with the predictions and variance and sorts: 1) high prediction first; 2) most important feature second (descending order) for overlapping predictions
preliminary = df_sorted.iloc[0:5] # Collects the first five columns
df_sorted2 = preliminary.sort_values(by=[keys2[-1], keys2[0]], ascending=[True, False]) # Sorts the top five rows by: 1) Low variance first; 2) most important feature second (descending order) for overlapping predictions
toPerform = df_sorted2.iloc[0] # First row is the selected reaction
toPerform

Unnamed: 0     23421.000000
Pyridine           0.300000
Aldehyde           0.200000
Isocyanide         0.300000
Temperature       40.000000
Solvent            0.100000
Catalyst           4.000000
Time              10.000000
Prediction         5.364171
Variance           2.437834
Name: 23411, dtype: float64

In [54]:
feat_imp.to_csv('feature_importances.txt', sep= '\t') 
best_params.to_csv('best_parameters.txt', sep= '\t')
toPerform.to_csv('selected_reaction.txt', sep = '\t')
df_sorted.to_csv('predictions.txt', sep = '\t')
filename3 = 'random_forest_model_grid.sav'
dump(grid, filename3)

['random_forest_model_grid.sav']